In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("JoinExample").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/15 19:38:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/15 19:39:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/15 19:39:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
26/01/15 19:39:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
emp_data = [
    (1, "Alice", 10),
    (2, "Bob", 20),
    (3, "Charlie", 30)
]

emp_columns = ["emp_id", "name", "dept_id"]

emp_df = spark.createDataFrame(emp_data, emp_columns)

emp_df.show()
emp_df.printSchema()


+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|     1|  Alice|     10|
|     2|    Bob|     20|
|     3|Charlie|     30|
+------+-------+-------+

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- dept_id: long (nullable = true)



In [3]:
dept_data = [
    (10, "HR"),
    (20, "IT"),
    (40, "Finance")
]

dept_columns = ["dept_id", "dept_name"]

dept_df = spark.createDataFrame(dept_data, dept_columns)

dept_df.show()
dept_df.printSchema()


+-------+---------+
|dept_id|dept_name|
+-------+---------+
|     10|       HR|
|     20|       IT|
|     40|  Finance|
+-------+---------+

root
 |-- dept_id: long (nullable = true)
 |-- dept_name: string (nullable = true)



INNER JOIN


👉 Show only matching records from both tables.

JOIN CONDITION
Meaning

👉 Rule that tells Spark how rows should match

In [11]:
emp_df.join(dept_df, emp_df.dept_id == dept_df.dept_id)

DataFrame[emp_id: bigint, name: string, dept_id: bigint, dept_id: bigint, dept_name: string]

In [6]:
emp_df.join(dept_df,emp_df.dept_id==dept_df.dept_id,"inner").show()

[Stage 6:==============================================>          (13 + 3) / 16]

+------+-----+-------+-------+---------+
|emp_id| name|dept_id|dept_id|dept_name|
+------+-----+-------+-------+---------+
|     1|Alice|     10|     10|       HR|
|     2|  Bob|     20|     20|       IT|
+------+-----+-------+-------+---------+



LEFT JOIN (LEFT OUTER)
Meaning

👉 Show all records from LEFT table
👉 Match from right if possible, else NUL

emp_df.join(dept_df,emp_df.dept_id==dept_df.dept_id,"left").show()

In [ ]:
RIGHT JOIN (RIGHT OUTER)
Meaning

👉 Show all records from RIGHT table
👉 Match from left if possible, else NULL

In [8]:
emp_df.join(dept_df,emp_df.dept_id==dept_df.dept_id,"right").show()

+------+-----+-------+-------+---------+
|emp_id| name|dept_id|dept_id|dept_name|
+------+-----+-------+-------+---------+
|     1|Alice|     10|     10|       HR|
|     2|  Bob|     20|     20|       IT|
|  NULL| NULL|   NULL|     40|  Finance|
+------+-----+-------+-------+---------+



FULL JOIN (FULL OUTER)
Meaning

👉 Show everything from both tables
👉 Match where possible, else NULL

In [9]:
emp_df.join(dept_df,emp_df.dept_id==dept_df.dept_id,"full").show()

+------+-------+-------+-------+---------+
|emp_id|   name|dept_id|dept_id|dept_name|
+------+-------+-------+-------+---------+
|     1|  Alice|     10|     10|       HR|
|     2|    Bob|     20|     20|       IT|
|     3|Charlie|     30|   NULL|     NULL|
|  NULL|   NULL|   NULL|     40|  Finance|
+------+-------+-------+-------+---------+



CROSS JOIN
Meaning

👉 Every row from left × every row from right
👉 No condition needed

⚠️ Can produce huge data

In [10]:
emp_df.crossJoin(dept_df).show()

[Stage 32:==========================================>          (105 + 16) / 131]

+------+-------+-------+-------+---------+
|emp_id|   name|dept_id|dept_id|dept_name|
+------+-------+-------+-------+---------+
|     1|  Alice|     10|     10|       HR|
|     1|  Alice|     10|     20|       IT|
|     1|  Alice|     10|     40|  Finance|
|     2|    Bob|     20|     10|       HR|
|     2|    Bob|     20|     20|       IT|
|     2|    Bob|     20|     40|  Finance|
|     3|Charlie|     30|     10|       HR|
|     3|Charlie|     30|     20|       IT|
|     3|Charlie|     30|     40|  Finance|
+------+-------+-------+-------+---------+



Multiple conditions

In [13]:
emp_df.join(
    dept_df,
    (emp_df.dept_id == dept_df.dept_id) &
    (emp_df.name.isNotNull()),
    "inner"
).show()

+------+-----+-------+-------+---------+
|emp_id| name|dept_id|dept_id|dept_name|
+------+-----+-------+-------+---------+
|     1|Alice|     10|     10|       HR|
|     2|  Bob|     20|     20|       IT|
+------+-----+-------+-------+---------+



In [14]:
emp_data = [
    (1, "Alice", 10),
    (2, "Bob", None),
    (3, "Charlie", 30)
]

emp_df = spark.createDataFrame(emp_data, ["emp_id", "name", "dept_id"])
emp_df.show()


+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|     1|  Alice|     10|
|     2|    Bob|   NULL|
|     3|Charlie|     30|
+------+-------+-------+



In [15]:
dept_data = [
    (10, "HR"),
    (20, "IT"),
    (30, "Finance")
]

dept_df = spark.createDataFrame(dept_data, ["dept_id", "dept_name"])
dept_df.show()


+-------+---------+
|dept_id|dept_name|
+-------+---------+
|     10|       HR|
|     20|       IT|
|     30|  Finance|
+-------+---------+



In [16]:
emp_df.join(dept_df, emp_df.dept_id == dept_df.dept_id, "inner").show()


+------+-------+-------+-------+---------+
|emp_id|   name|dept_id|dept_id|dept_name|
+------+-------+-------+-------+---------+
|     1|  Alice|     10|     10|       HR|
|     3|Charlie|     30|     30|  Finance|
+------+-------+-------+-------+---------+



Bob missing because:

NULL = NULL ❌ (never matches)

In [17]:
emp_df.join(dept_df, emp_df.dept_id == dept_df.dept_id, "left").show()

+------+-------+-------+-------+---------+
|emp_id|   name|dept_id|dept_id|dept_name|
+------+-------+-------+-------+---------+
|     1|  Alice|     10|     10|       HR|
|     2|    Bob|   NULL|   NULL|     NULL|
|     3|Charlie|     30|     30|  Finance|
+------+-------+-------+-------+---------+



Replace NULL before join (Best practice)

In [18]:
emp_df_filled = emp_df.fillna({"dept_id": -1})
dept_df_filled = dept_df.fillna({"dept_id": -1})

emp_df_filled.join(
    dept_df_filled,
    emp_df_filled.dept_id == dept_df_filled.dept_id,
    "left"
).show()


+------+-------+-------+-------+---------+
|emp_id|   name|dept_id|dept_id|dept_name|
+------+-------+-------+-------+---------+
|     1|  Alice|     10|     10|       HR|
|     2|    Bob|     -1|   NULL|     NULL|
|     3|Charlie|     30|     30|  Finance|
+------+-------+-------+-------+---------+



dropna


👉 Remove rows that contain NULL values

Drop rows where ANY column is NULL

In [19]:
emp_df.dropna().show()

+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|     1|  Alice|     10|
|     3|Charlie|     30|
+------+-------+-------+



Drop rows where specific column is NULL

In [20]:
emp_df.dropna(subset=["dept_id"]).show()

+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|     1|  Alice|     10|
|     3|Charlie|     30|
+------+-------+-------+



Drop only if ALL columns are NULL

In [21]:
emp_df.dropna(how="all").show()


+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|     1|  Alice|     10|
|     2|    Bob|   NULL|
|     3|Charlie|     30|
+------+-------+-------+



fillna

👉 Replace NULL values with given value

In [22]:
emp_df.fillna({"dept_id": 0}).show()


+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|     1|  Alice|     10|
|     2|    Bob|      0|
|     3|Charlie|     30|
+------+-------+-------+



In [23]:
emp_df.fillna({
    "dept_id": 0,
    "name": "Unknown"
}).show()


+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|     1|  Alice|     10|
|     2|    Bob|      0|
|     3|Charlie|     30|
+------+-------+-------+



In [24]:
emp_df.fillna("NA").show()


+------+-------+-------+
|emp_id|   name|dept_id|
+------+-------+-------+
|     1|  Alice|     10|
|     2|    Bob|   NULL|
|     3|Charlie|     30|
+------+-------+-------+



In [26]:
dept_df.fillna("NA").show()


+-------+---------+
|dept_id|dept_name|
+-------+---------+
|     10|       HR|
|     20|       IT|
|     30|  Finance|
+-------+---------+

